In [1]:
# $example on$
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# $example off$
from pyspark.sql import SparkSession


from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.ml import Pipeline

from pyspark.ml.feature import VectorAssembler

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel

import csv
from pyspark.mllib.regression import LabeledPoint

In [2]:
rdd = sc.textFile("processed2.csv")

In [3]:
temp1 = rdd.map(lambda row: row.split(','))
temp2 = temp1.map(lambda seq: LabeledPoint(seq[0],seq[1:])) 

In [4]:
(trainingData, testData) = temp2.randomSplit([0.7, 0.3])

In [14]:
model = GradientBoostedTrees.trainRegressor(trainingData,categoricalFeaturesInfo={}, numIterations=1)

In [15]:
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression GBT model:')
print(model.toDebugString())

Test Mean Squared Error = 0.165014030596
Learned regression GBT model:
TreeEnsembleModel regressor with 1 trees

  Tree 0:
    If (feature 45 <= -0.0017897811)
     If (feature 16 <= 5.743573E-4)
      If (feature 70 <= 0.0034970013)
       Predict: 4.383977770714252
      Else (feature 70 > 0.0034970013)
       Predict: 4.5163728666330005
     Else (feature 16 > 5.743573E-4)
      If (feature 270 <= -0.0013743711)
       Predict: 4.179680040251923
      Else (feature 270 > -0.0013743711)
       Predict: 4.390508968642237
    Else (feature 45 > -0.0017897811)
     If (feature 235 <= 3.214122E-4)
      If (feature 263 <= 0.0036243114)
       Predict: 4.345561778919556
      Else (feature 263 > 0.0036243114)
       Predict: 4.477342771633922
     Else (feature 235 > 3.214122E-4)
      If (feature 251 <= -0.0011957733)
       Predict: 4.342845047197716
      Else (feature 251 > -0.0011957733)
       Predict: 4.2880171426959475



In [9]:
model.save(sc, "SGBT_1")
#sameModel = GradientBoostedTreesModel.load(sc, "SGBT_1")
   